In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

def download_liar_dataset():
    """
    Function to download the LIAR dataset.

    Note: The LIAR dataset needs to be downloaded manually from:
    https://www.cs.ucsb.edu/~william/data/liar_dataset.zip

    After downloading, unzip the file and place the train, test and valid files
    in a directory called 'liar_dataset'.

    Returns:
        train_df, test_df, valid_df (tuple of pandas DataFrames)
    """
    # Define column names for the datasets
    column_names = [
        'id', 'label', 'statement', 'subject', 'speaker', 'speaker_job',
        'state_info', 'party', 'barely_true_counts', 'false_counts',
        'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'
    ]

    try:
        # Load datasets
        train_df = pd.read_csv('liar_dataset/train.tsv', sep='\t', header=None, names=column_names)
        test_df = pd.read_csv('liar_dataset/test.tsv', sep='\t', header=None, names=column_names)
        valid_df = pd.read_csv('liar_dataset/valid.tsv', sep='\t', header=None, names=column_names)

        print(f"Successfully loaded dataset: {len(train_df)} training, {len(test_df)} test, {len(valid_df)} validation examples")
        return train_df, test_df, valid_df

    except FileNotFoundError:
        print("Dataset files not found. Please download the LIAR dataset from:")
        print("https://www.cs.ucsb.edu/~william/data/liar_dataset.zip")
        print("After downloading, unzip and place the files in a 'liar_dataset' directory.")
        return None, None, None

def preprocess_text(text):
    """
    Function to preprocess text data.

    Args:
        text (str): Input text to preprocess

    Returns:
        str: Preprocessed text
        list: Tokens
        list: Stemmed tokens
        list: Lemmatized tokens
    """
    #checks if text is nan
    if pd.isna(text):
        return "", [], [], []

    text = text.lower()

    #removes URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    #removes HTML tags
    text = re.sub(r'<.*?>', '', text)

    #removes special characters/numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    #tokenize
    tokens = word_tokenize(text)

    #removes stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    #stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    #lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    #reconstruct the preprocessed text
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text, tokens, stemmed_tokens, lemmatized_tokens

def extract_sentiment_features(text):
    """
    Function to extract sentiment features from text.

    Args:
        text (str): Input text

    Returns:
        dict: Dictionary containing sentiment features
    """
    #check if text is empty
    if not text:
        return {
            'polarity': 0,
            'subjectivity': 0,
            'compound': 0,
            'pos': 0,
            'neu': 0,
            'neg': 0
        }

    #sentiment analysis (textblob)
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity

    #sentiment analysis (VADER)
    sid = SentimentIntensityAnalyzer()
    vader_scores = sid.polarity_scores(text)

    return {
        'polarity': polarity,
        'subjectivity': subjectivity,
        'compound': vader_scores['compound'],
        'pos': vader_scores['pos'],
        'neu': vader_scores['neu'],
        'neg': vader_scores['neg']
    }

def process_liar_dataset():
    """
    Main function to process the LIAR dataset.

    Returns:
        pd.DataFrame: Processed dataset with features
    """
    #download and load the dataset
    train_df, test_df, valid_df = download_liar_dataset()

    if train_df is None:
        return None

    #combines datasets for processing
    combined_df = pd.concat([train_df, test_df, valid_df], ignore_index=True)

    #list to store processed data
    processed_data = []

    #process each statement
    for idx, row in combined_df.iterrows():
        statement = row['statement']

        #preprocess text
        preprocessed_text, tokens, stemmed_tokens, lemmatized_tokens = preprocess_text(statement)

        #extracting sentiment features
        sentiment_features = extract_sentiment_features(preprocessed_text)

        #stores processed data
        processed_item = {
            'id': row['id'],
            'label': row['label'],
            'statement': statement,
            'preprocessed_text': preprocessed_text,
            'num_tokens': len(tokens),
            'speaker': row['speaker'],
            'context': row['context'],
            **sentiment_features  # Unpack sentiment features
        }

        processed_data.append(processed_item)

        #printing the progress every 500 items
        if (idx + 1) % 500 == 0:
            print(f"Processed {idx + 1}/{len(combined_df)} statements")

    #creating DataFrame from processed data
    processed_df = pd.DataFrame(processed_data)

    #saving processed data
    processed_df.to_csv('liar_processed.csv', index=False)
    print(f"Processed dataset saved to 'liar_processed.csv' with {len(processed_df)} records")

    return processed_df

def analyze_sentiment_distribution(df):
    """
    Analyze sentiment distribution by label.

    Args:
        df (pd.DataFrame): Processed dataset

    Returns:
        pd.DataFrame: Sentiment statistics by label
    """
    #group by label/calculate mean sentiment scores
    sentiment_by_label = df.groupby('label')[['polarity', 'subjectivity', 'compound', 'pos', 'neu', 'neg']].mean()

    print("\nSentiment Analysis by Label:")
    print(sentiment_by_label)

    return sentiment_by_label

if __name__ == "__main__":
    print("Starting LIAR dataset processing...")
    processed_df = process_liar_dataset()

    if processed_df is not None:
        #print dataset info
        print("\nDataset Information:")
        print(processed_df.info())

        #print label distribution
        print("\nLabel Distribution:")
        print(processed_df['label'].value_counts())

        #analyze sentiment distribution
        sentiment_stats = analyze_sentiment_distribution(processed_df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Starting LIAR dataset processing...
Successfully loaded dataset: 10240 training, 1267 test, 1284 validation examples
Processed 500/12791 statements
Processed 1000/12791 statements
Processed 1500/12791 statements
Processed 2000/12791 statements
Processed 2500/12791 statements
Processed 3000/12791 statements
Processed 3500/12791 statements
Processed 4000/12791 statements
Processed 4500/12791 statements
Processed 5000/12791 statements
Processed 5500/12791 statements
Processed 6000/12791 statements
Processed 6500/12791 statements
Processed 7000/12791 statements
Processed 7500/12791 statements
Processed 8000/12791 statements
Processed 8500/12791 statements
Processed 9000/12791 statements
Processed 9500/12791 statements
Processed 10000/12791 statements
Processed 10500/12791 statements
Processed 11000/12791 statements
Processed 11500/12791 statements
Processed 12000/12791 statements
Processed 12500/12791 statements
Processed dataset saved to 'liar_processed.csv' with 12791 records

Dataset In